# Đề tài: **Mercari Price Suggestion Challenge**

# 1.Mô tả bài toán

- Ngày nay,  rất khó để biết chính xác giá bán của các cản phẩm. Các sản phẩm chỉ khác nhau một vài chi tiết nhỏ cũng có thể có giá khác nhau rất lớn
- Bài toán cuộc thi này đặt ra là: xây dựng một thuật toán mà tự động đề xuất giá sản phẩm một cách chính xác. Chúng ta được cung cấp các miêu tả về sản phẩm như thông tin về **tên danh mục, tên thương hiệu và tình trạng mặt hàng**

# 2.Giải Nén

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

# 3.Đọc dữ liệu
Đọc dữ liệu từ 2 tập 'train.tsv' và tập 'test_stg2.tsv'

In [ ]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test_stg2.tsv', sep='\t')

In [ ]:
print ("Train data shape:", train.shape)
print ("Test data shape:", test.shape)

In [ ]:
#Quan sát dữ liệu ở tập train
#Sử dụng dữ liệu ở tập train để huấn luyện mô hình
train.head(10)

In [ ]:
#Quan sát dữ liệu ở tập test
#Sử dụng dữ liệu ở tập test để kiểm tra 
test.head(10)

 **Quan sát ban đầu**
* Mỗi 1 sản phấm sẽ có các thuộc tính: train_id, name, item_condition_id, category_name, shipping, item_description 
* Tập train có thêm cột price
* Ở cột brand_name có nhiều giá trị NaN(giá trị rỗng)

# 4.Chuẩn bị thư viện

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
import scipy
from scipy.sparse import csr_matrix, hstack
import gc

# 5.Phân tích dữ liệu

**Mục tiêu**: Ở phần này chúng ta sẽ xử lí các dữ liệu. Vì bài toán này là bài toán hồi quy tuyến tính nên cần chuyển các thuộc tính về dạng vector hoặc số thực

### 5.1 Kiểu dữ liệu của các tập

In [ ]:
#Kiểm tra thông tin của tập 'train'
train.info()

In [ ]:
#Kiểm tra thông tin dữ liệu ở tập 'test'
test.info()

In [ ]:
train.nunique()

### 5.2 Đếm số lượng ô trống trong 2 tập train, test

In [ ]:
#Số lượng các vị trí không có giá trị trong tập train
train.isnull().sum()

In [ ]:
#Số lượng các vị trí không có giá trị trong tập test
test.isnull().sum()

=> Các trường thuộc tính có giá trị null là category_name,  brand_name và item_description

In [ ]:
print("{0:.2f}% sản phẩm không có thương hiệu(brand_name) ở tập train".format((pd.isnull(train['brand_name']).sum())*100/train.shape[0]))

print("{0:.2f}% sản phẩm không có phân loại(category_name) ở tập train".format((pd.isnull(train['category_name']).sum())*100/train.shape[0]))
print("{0:.10f}% sản phẩm không có mô tả(item_description) ở tập train".format((pd.isnull(train['item_description']).sum())*100/train.shape[0]))

In [ ]:
print("{0:.2f}% sản phẩm không có thương hiệu(brand_name) ở tập test".format((pd.isnull(test['brand_name']).sum())*100/test.shape[0]))
print("{0:.2f}% sản phẩm không có phân loại(category_name) ở tập test".format((pd.isnull(test['category_name']).sum())*100/test.shape[0]))
print("{0:.10f}% sản phẩm không có mô tả(item_description) ở tập test".format((pd.isnull(test['item_description']).sum())*100/test.shape[0]))

**Quan sát dữ liệu ở trên ta thấy được:**
* Có 3 tập có giá trị null
* brand_name có gần một nửa giá trị bị thiếu cả trong tập train và tập test, vì vậy không thể xóa những hàng mà phải lấp các giá trị đó
* Category_name thiếu 0,43-0,44% giá trị nên có thể xóa các hàng đó hoặc lấp đầy các giá trị đó
* Tương tự với item_description có thể xóa hoặc lấp đầy các hàng đó

### 5.3 Dữ liệu dạng số

In [ ]:
#Định dạng 
pd.set_option('display.float_format', lambda x: '%.5f' % x)
#Thống kê số lượng, giá trị trung bình, min, max, giá trị phổ biến của các cột chứa dữ liệu dạng số
train.describe()

**Quan sát**
* Ở đây chỉ có 2 cột dữ liệu dùng để phân tích dự đoán là cột item_condition_id và shipping
* Giá trị của các mặt hàng nhỏ nhất là không tức là có những sản phẩm có thể xem như là miễn phí và lớn nhất là 2009
* Giá trị phổ biến của các sản phẩm dao động từ 17 − 2009
* Phần lớn các mặt hàng có shipping = 1

### 5.4 Quan sát và đánh giá dữ liệu ở cột item_condition_id

In [ ]:
#Biểu đồ phân tích cột item_condition_id
ax = sns.countplot(x = 'item_condition_id',data=train, palette ='Blues_r')
ax.set_title("Tổng số lượng sản phẩm theo item_condition_id", fontsize = 13)

****Quan sát****

* Có 5 loại tình item_condition_id
* Loại 1, loại 2 và loại 3 là phổ biến nhất
* Loại 4 và loại 5 là ít phổ biến nhất

### 5.5 Đánh giá cột price

In [ ]:
train = train.drop(train[(train['price'] >= 200)].index, axis=0)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.histplot(train['price'], bins=50, ax=ax)
plt.show()

Chúng ta có thể thấy rằng phân phối ở trên bị lệch về bên trái nên ta sẽ đưa về phân phối chuẩn với log(price)

In [ ]:
train['log_price'] = np.log1p(train['price'])

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.histplot(train['log_price'], bins=50, ax=ax)
plt.show()

Phân bố của log(price) ít lệch hơn và được phân bố tốt xung quanh giá trị trung bình
=> Sử dụng log(price) thay vì price để huấn luyện

In [ ]:
train.head(5)

### 5.6 Đánh giá cột shipping

In [ ]:
(train['shipping'].value_counts())*100/train.shape[0]

=> Có khoảng 55% khách hàng không phải trả phí vận chuyển

In [ ]:
plt.figure(figsize=(10,8))

plt.hist(np.log(train['price'][train['shipping']==0]+1),bins=50, edgecolor='white',color="blue", label='shipping = 0') 
plt.hist(np.log(train['price'][train['shipping']==1]+1),bins=50, edgecolor='white',color="cornflowerblue",label="shipping = 1") 
plt.xlabel('log_Price')
plt.legend(loc='upper right')
plt.title('Shipping')
plt.show()

### 5.7 Quan sát dữ liệu cột Brand_name

In [ ]:
#Danh sách 10 thương hiệu phổ biến nhất
#(dữ liệu thương hiệu chưa được lấp đầy các giá trị trống)
brands =train['brand_name'].value_counts()
print(brands[:10])

**Quan sát:**
* Các thương hiệu phổ biến nhất là: PINK, Nike, Victoria's Secret, LuLaRoe, Apple, FOREVER 21, Nintendo, Lululemon, Michael Kors,American Eagle  .

In [ ]:
# Đếm số lượng 1 số từ trong 'name'
train['name'].value_counts()[:10]

In [ ]:
#Quan sát phân phối giá của các thương hiệu phổ biến nhất
plt.figure(figsize=(20, 15))
plt.subplot(3, 3, 1)
plt.hist(np.log(train['price'][train['brand_name']=='Nike']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Nike')

plt.subplot(3, 3, 2)
plt.hist(np.log(train['price'][train['brand_name']=="Victoria's Secret"]+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title("Victoria's Secret")

plt.subplot(3, 3, 3)
plt.hist(np.log(train['price'][train['brand_name']=='LuLaRoe']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('LuLaRoe')

**Quan sát**
* Phân phối của log(price) của các thương hiệu phổ biến và các sản phẩm không có thương hiệu đều tuân theo phân phối chuẩn

In [ ]:
#một số thương hiệu ít phổ biến hơn
print(brands[250:260])

In [ ]:
plt.figure(figsize=(20, 15))

plt.subplot(3, 3, 1)
plt.hist(np.log(train['price'][train['brand_name']=='Leap Frog']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Leap Frog')

plt.subplot(3, 3, 2)
plt.hist(np.log(train['price'][train['brand_name']=='Gerber']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Gerber')

plt.subplot(3, 3, 3)
plt.hist(np.log(train['price'][train['brand_name']=='Madewell']+1),bins=50, edgecolor='white')
plt.xlabel('log_Price')
plt.ylabel('Frequency')
plt.title('Madewell')


**Quan sát**
* Đa số các phân phối log_Price của các hãng này đều tuân theo phân phối chuẩn

### 5.7 Quan sát dữ liệu cột 'name'

In [ ]:
x = train['name'].apply(lambda x: len(x))
plt.hist(x,bins = 30,range=[0,50],edgecolor='white')
plt.show()

**Quan sát:**

* Tên của mỗi sản phẩm có khoảng từ 1 đến 42 từ và chủ yếu nẳm trong khoảng từ 15 đến 40 từ

### 5.8 Quan sát dữ liệu cột 'item_description'

In [ ]:
from wordcloud import WordCloud
import os
wordcloud = WordCloud(width = 2400, height = 1200).generate(" ".join(train.item_description.astype(str)))
plt.figure(figsize = (13, 10))
plt.imshow(wordcloud)
plt.show()

=> Các cụm từ "Brand new", "free shipping", "great condition", " good condition", "never worn", "smoke free", " description yet" là những cụm từ xuất hiện nhiều nhất

## 6.PreProcess

In [ ]:
train_row = train.shape[0]
test_id = test['test_id']

### 6.1 Loại bỏ cột price trong dataset và lấy log(price) để test

In [ ]:
target = np.log1p(train['price'])
train = train[[col for col in train.columns if col != 'price']]

print(train.shape)
print(target.shape)

In [ ]:
train.head()

### 6.2 Xử lý các ô trống

In [ ]:
#Nối tập train, test để tiền xử lý trên cả 2 tập
X = pd.concat([train, test], axis=0)

print(X.shape)
X.head()

In [ ]:
#Số lượng ví trí khôg có giá trị
X.isnull().sum()

***Lấp tất cả các giá trị null***

In [ ]:
X.brand_name.fillna(value = "NoBrand", inplace = True)

X.category_name.fillna(value = "Other/Other/Other", inplace = True)

X.item_description.fillna(value = "No description yet", inplace = True)


In [ ]:
X.isnull().sum()

In [ ]:
#danh sách 10 thương hiệu phổ biến nhất
brands = X['brand_name'].value_counts()
print(brands[:10])

**Quan sát:**
* Những sản phẩm không có thương hiệu chiếm số lượng nhiều nhất.
* Các thương hiệu phổ biến nhất là: PINK, Nike, Victoria's Secret, LuLaRoe, Apple, FOREVER 21, Nintendo, Lululemon, Michael Kors.

### Vector hóa cột 'name'
* Mỗi một chuỗi ở cột "name" khá ngắn
* Sử dụng CountVectorizer() để chuyển cột name từ dạng text về dạng một vecto trên cơ sở số lần xuất hiện của mỗi từ
* CountVectorizer tạo một ma trận trong đó mỗi từ duy nhất được biểu thị bằng một cột của ma trận và với tên của mỗi sản phẩm là một hàng trong ma trận. Giao của một hàng và một cột chính là số lần xuất hiện của từ(tương úng với cột) trong tên của sản phẩm (tương úng với hàng)

In [ ]:
cv = CountVectorizer()

name_data = cv.fit_transform(X['name'])
name_data

In [ ]:
print(name_data.shape)

=> ma trận 'name_data'có 4932413 hàng là vì có 4932413 sản phẩm trong tập 'X' và có 207463 tức là có tất cả 207463 từ xuất hiện trong cột 'name'

### Vector hóa trường item_description
* Do số lượng từ ở cột item_description nhiều => sử dụng TfidfVectorizer() để chuyển đổi dữ liệu thành dạng ma trận
* TF-IDF (Term Frequency – Inverse Document Frequency) là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản.
- Tần suất từ (term frequency - tf):
    
    $$
    \mathrm{tf}_{t, d} = \begin{cases}1+\log_{10} count(t, d)&\textrm{nếu } count(t, d) > 0\\ 0 &\textrm{ngược lại}\end{cases}
    $$
    
    với $count(t,d)$ là số lần xuất hiện của từ $t$ trong văn bản (lân cận) $d$.
    
- Nghịch đảo tần suất văn bản (inverse document frequency - idf):
    
    $$
    \mathrm{idf}_t = \log\frac N {\mathrm{df}_t}
    $$
    
    với $N$ là số lượng văn bản, $\mathrm{df}_t$ là số văn bản có từ $t$.
    

Tích của hai thành phần này cho ta mã hoá của từ $t$ trong văn bản $d$.

$$
w_{t,d} = \mathrm{tf}_{t, d} \times \mathrm{idf}_t
$$

Như vậy ngữ cảnh của một từ được mã hoá thành vector thuộc $\mathbb R^{n_V}$ các hệ số tf-idf của các từ trong ngữ cảnh đó (thay cho số đếm). Các vector này thường là các vector thưa (đa phần là số 0).

In [ ]:
tv = TfidfVectorizer()

description_data = tv.fit_transform(X['item_description'])
description_data

In [ ]:
description_data.shape

### Vector hóa các trường 'brand_name' và 'category_name' bằng LabelBinarizer
- Sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân
- Kết quả trả về là một vector gồm các chuỗi đã được mã hóa theo: chuỗi 
$y_i$ thì sẽ là vector chứa toàn số 0, có duy nhất 1 số 1 ở vị trí $y_i$ (lưu ý các chuỗi được sắp xếp theo thứ tự theo bảng chữ cái trước)

In [ ]:
lb = LabelBinarizer(sparse_output=True)
brand_data = lb.fit_transform(X['brand_name'])
brand_data

In [ ]:
brand_data.shape

=> ma trận trả về có 4932413 hàng (vì trong tập X có dữ liệu của 4932413 sản phẩm) và có 6309 cột vì có 6309 thương hiệu

In [ ]:
category_data = lb.fit_transform(X['category_name'])
category_data

### Sử dụng **scipy.sparse.csr_matrix** và **get_dummy** để mã hóa **shipping** và **item_condition_id**

In [ ]:
condition_data = scipy.sparse.csr_matrix(pd.get_dummies(X['item_condition_id'], sparse = True).values)
condition_data

=> ma trận trả về có 4932413 hàng (vì trong tập train có dữ liệu của 4932413 sản phẩm) và có 5 cột vì chỉ có 5 kiểu item_condition

In [ ]:
shipping_data = scipy.sparse.csr_matrix(pd.get_dummies(X['shipping'], sparse = True).values)
shipping_data

=> ma trận trả về có 4932413 hàng (vì trong tập train có dữ liệu của 4932413 sản phẩm) và có 1 cột vì chỉ có 2 kiểu shipping

# Model

**Tạo sparse matrix để kết hợp các dữ liệu với nhau**

In [ ]:
X_sparse = scipy.sparse.hstack((name_data, brand_data, description_data, category_data, condition_data, shipping_data)).tocsr()
X_sparse

In [ ]:
train = X_sparse[:train_row]
test = X_sparse[train_row:]

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
del X, X_sparse, name_data, description_data, brand_data, category_data, condition_data, shipping_data, train_row, cv, tv, lb


In [ ]:
gc.collect()

### Mô hình LGBMRegressor
- **LGBMRegressor**
    - LightBGM ra đời đã giúp chúng ta giải bài toán về thời gian traing lâu đối với bộ dữ liệu lớn của phương pháp **XGBoost** và trở thành **thuật toán ensemble được ưa chuộng nhất**
    - LGBM trở thành **thuật toán ensemble được ưa chuộng nhất** vì**:**
        - LightGBM sử dụng "**histogram-based algorithms**" thay thế cho "pre-sort-based algorithms " thường được dùng trong các boosting tool khác để tìm kiếm split point trong quá trình xây dựng tree. Cải tiến này giúp LightGBM tăng tốc độ training, đồng thời làm giảm bộ nhớ cần sử dụng
        - LightGBM phát triển tree dựa trên **leaf-wise**, trong khi hầu hết các boosting tool khác (kể cả xgboost) dựa trên level (depth)-wise. Leaf-wise lựa chọn nút để phát triển cây dựa trên tối ưu toàn bộ tree, trong khi level-wise tối ưu trên nhánh đang xét, do đó, với số node nhỏ, **các tree xây dựng từ leaf-wise thường out-perform level-wise**.
        - Thật ra cả xgboost và lightgbm đều sử dụng histogram-based algorithms, điểm tối ưu của lightgbm so với xgboost là ở 2 thuật toán: **GOSS (Gradient Based One Side Sampling) và EFB (Exclusive Feature Bundling)** giúp tăng tốc đáng kể trong quá trình tính toán (chi tiết ta sẽ trình bày ở dưới)
        - **GOSS (Gradient Based One Side Sampling)**
            - Là một phương pháp lấy mẫu mới, lấy mẫu dựa trên độ dốc. GOSS lưu giữ tất cả các trường hợp có độ dốc lớn và thực hiện lấy mẫu ngẫu nhiên trên các trường hợp có độ dốc nhỏ. Để bù đắp ảnh hưởng đến việc phân phối dữ liệu, khi tính toán, GOSS đưa vào hệ số nhân không đổi cho các trường hợp dữ liệu có đạo hàm nhỏ
        - **EFB (Exclusive Feature Bundling)**
            - Là một phương pháp gần như không mất dữ liệu để giảm số lượng đặc trưng hiệu quả. Trong một không gian đặc trưng thưa thớt, nhiều đặc trưng gần như là độc quyền, ngụ ý rằng chúng hiếm khi nhận các giá trị khác không đồng thời. EFB kết hợp các đặc trưng này, giảm kích thước để cải thiện hiệu quả trong khi vẫn duy trì mức độ chính xác cao.
        
        - **Architecture of LightGBM**
            - Như đã nói ở trên LBGM phát triển tree dựa trên **leaf-wise,** nó lựa chọn là có lỗi lớn nhất để xử lí. Do lá được cố định nên thuật toán sẽ có lỗi ít hơn so với **level-wise.**
            - Note: *Leaf-wise tuy tốt, nhưng với những bộ dữ liệu nhỏ, các tree xây dựng dựa trên leaf-wise thường dẫn đến **overfit khá sớm**. Do đó, lightgbm sử dụng thêm 1 hyperparameter là maxdepth nhằm cố gắng hạn chế điều này. Dù vậy, LightGBM vẫn được khuyến khích sử dụng khi bộ dữ liệu là đủ lớn.*

In [ ]:
import lightgbm as lgb

In [ ]:
params = {'num_leaves': 19,
          'n_estimators': 294,
          'learning_rate': 0.30629521899319057,
          'max_depth': 8,
          'min_child_samples': 836,
          'min_data_in_leaf': 32,
          'bagging_freq': 4,
          'bagging_fraction': 0.1460764432004789,
          'feature_fraction': 0.8707981099370118,
          'subsample': 0.3296674411363768,
          'colsample_bytree': 0.8999093468212471,
          'random_state': 666}

In [ ]:
cls = lgb.LGBMRegressor(**params)
cls.fit(train, target)

In [ ]:
#Tính rmsle
def rmsle(y, y_preds):
    assert len(y) == len(y_preds)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_preds), 2)))

###  Số liệu đánh giá : RMSLE
* Số liệu đánh giá cuộc thì này là Root Mean Squared Logarithmic Error
* Lỗi trung bình bình phương (RMSE) là độ lệch chuẩn của phần dư ( lỗi dự đoán ). Phần dư là thước đo khoảng cách từ các điểm dữ liệu đường hồi quy; RMSE là thước đo mức độ lan truyền của những phần dư này. Nói cách khác, nó cho bạn biết mức độ tập trung của dữ liệu xung quanh dòng phù hợp nhất . Lỗi bình phương trung bình thường được sử dụng trong khí hậu học, dự báo và phân tích hồi quy để xác minh kết quả thí nghiệm.
* Công thức tính RMSLE


![](https://thienmaonline.vn/rmse-la-gi/imager_5_11689_700.jpg)


Trong đó
* y^i là giá trị ước lượng
* yi là biến độc lập
* n=(N – k – 1)
* N : số tổng lượng quan sát
* K : tổng lượng biến

In [ ]:
pred = cls.predict(train)
rmsle(target, pred)

In [ ]:
del train, target, params
gc.collect()

# Submit

In [ ]:
prediction = np.expm1(cls.predict(test))

In [ ]:
prediction

In [ ]:
del test
gc.collect()

In [ ]:
submit = pd.DataFrame(test_id, columns=['test_id'])
submit['price'] = prediction

submit.head(10)

In [ ]:
del test_id, prediction
gc.collect()

In [ ]:
submit.to_csv('./submission.csv', index=False)